# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [11]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [12]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [58]:
symbol = 'AAL'
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=advanced-stats,price&token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
data

{'AAL': {'advanced-stats': {'beta': 1.5996540194943165,
   'totalCash': 7077226847,
   'currentDebt': 16729019277,
   'revenue': 17447325283,
   'grossProfit': 17341492874,
   'totalRevenue': 17552716451,
   'EBITDA': -9156801625,
   'revenuePerShare': 27.8,
   'revenuePerEmployee': 132246.79,
   'debtToEquity': -9.209132486302218,
   'profitMargin': -0.519694401113439,
   'enterpriseValue': 40715158351,
   'enterpriseValueToRevenue': 2.34,
   'priceToSales': 0.8767,
   'priceToBook': -2.23,
   'forwardPERatio': -2.990778477127401,
   'pegRatio': 0.0029051283720664114,
   'peHigh': 9.790906199005406,
   'peLow': 6.5401306587926,
   'week52highDate': '2021-03-13',
   'week52lowDate': '2020-05-02',
   'putCallRatio': 0.4465339212901846,
   'companyName': 'American Airlines Group Inc',
   'marketcap': 15246352374,
   'week52high': 27.21,
   'week52low': 8.29,
   'week52highSplitAdjustOnly': 26.69,
   'week52highDateSplitAdjustOnly': '2021-03-05',
   'week52lowSplitAdjustOnly': 8.26,
   'w

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [14]:
price = data['latestPrice']
print(price)

peRatio = data['peRatio']
print(peRatio)

136.65
36.66


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [30]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [35]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],index=my_columns
            ),ignore_index=True
        )
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,136.39,51.65,N/A
1,AAL,23.71,-1.28,N/A
2,AAP,196.20,26.36,N/A
3,AAPL,132.22,37.16,N/A
4,ABBV,112.46,41.2,N/A
...,...,...,...,...
500,YUM,117.99,39.7,N/A
501,ZBH,173.34,-251.43,N/A
502,ZBRA,514.90,55.98,N/A
503,ZION,58.60,9.02,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [36]:
final_dataframe.sort_values('Price-to-Earnings Ratio',ascending=True,inplace=True) #inplace ensures that the original dataframe is modified
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis=1,inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,628.470,4.86,N/A
1,ALL,119.510,6.9,N/A
2,UNM,29.910,7.62,N/A
3,EBAY,64.980,8.15,N/A
4,AFL,54.120,8.18,N/A
5,COO,401.290,8.82,N/A
6,KIM,19.690,8.93,N/A
7,ZION,58.600,9.02,N/A
8,CE,154.980,9.33,N/A
9,CTL,11.000,9.86,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [37]:
def portfolio_input():
    global portfolio_size
    flag=0
    while(flag == 0):
        try:
            portfolio_size = input('Enter the value of your portfolio: ')
            val = float(portfolio_size)
            flag=1
        except ValueError:
            print("That is not a number\nPlease try again")
    portfolio_size = val
    print(val)

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [38]:
portfolio_input()

Enter the value of your portfolio: 10000000
10000000.0


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [44]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
     final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,628.470,4.86,318
1,ALL,119.510,6.9,1673
2,UNM,29.910,7.62,6686
3,EBAY,64.980,8.15,3077
4,AFL,54.120,8.18,3695
5,COO,401.290,8.82,498
6,KIM,19.690,8.93,10157
7,ZION,58.600,9.02,3412
8,CE,154.980,9.33,1290
9,CTL,11.000,9.86,18181


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [134]:
hqv_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'Enterprise value',
    'EBITDA',
    'Gross profit',
    'Price-to-earnings ratio',
    'PE percentile',
    'Price-to-book ratio',
    'PB percentile',
    'Price-to-sales ratio',
    'PS percentile',
    'EV/EBITDA',
    'EV/EBITDA percentile',
    'EV/GP',
    'EV/GP percentile',
    'Robust Value score'
]

hqv_dataframe = pd.DataFrame(columns=hqv_columns)


for symbol_string in symbol_strings:
    
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=advanced-stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        grossProfit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebida = enterprise_value/ebitda
        except TypeError:
            ev_to_ebida = np.NaN
        
        try:
            ev_to_grossProfit = enterprise_value/grossProfit
        except TypeError:
            ev_to_grossProfit = np.NaN
        
        
        hqv_dataframe = hqv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['advanced-stats']['enterpriseValue'],
                    data[symbol]['advanced-stats']['EBITDA'],
                    data[symbol]['advanced-stats']['grossProfit'],
                    data[symbol]['advanced-stats']['forwardPERatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebida,
                    'N/A',
                    ev_to_grossProfit,
                    'N/A',
                    'N/A'
                    
                ],index=hqv_columns
            ),ignore_index=True
        )
hqv_dataframe

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,A,137.53,N/A,41480769866,1292974812,3081228017,34.651666,N/A,8.63,N/A,7.57,N/A,32.081653,N/A,13.462415,N/A,N/A
1,AAL,23.09,N/A,41304382486,-9134633813,18041471728,-2.975949,N/A,-2.24,N/A,0.8617,N/A,-4.521734,N/A,2.289413,N/A,N/A
2,AAP,187.02,N/A,13104421376,1048679617,4537566971,19.230496,N/A,3.55,N/A,1.23,N/A,12.496115,N/A,2.887984,N/A,N/A
3,AAPL,139.23,N/A,2251326217162,85513362346,117593467946,30.51562,N/A,33.83,N/A,7.75,N/A,26.327186,N/A,19.144994,N/A,N/A
4,ABBV,112.45,N/A,269779890488,19418945818,30762984954,None,N/A,15.12,N/A,4.19,N/A,13.892613,N/A,8.769627,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.78,N/A,46862684256,1785040243,4257681238,30.308862,N/A,-4.6,N/A,6.44,N/A,26.253013,N/A,11.006621,N/A,N/A
501,ZBH,175.50,N/A,43304953350,1398920418,5026018529,22.093283,N/A,2.99,N/A,5.21,N/A,30.955981,N/A,8.616155,N/A,N/A
502,ZBRA,508.99,N/A,29299037721,781962872,2102234579,32.805023,N/A,12.69,N/A,6.23,N/A,37.468579,N/A,13.937092,N/A,N/A
503,ZION,56.59,N/A,9493555781,1313880465,3148618467,12.860578,N/A,1.31,N/A,2.78,N/A,7.225586,N/A,3.015150,N/A,N/A


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [135]:
hqv_dataframe[hqv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
4,ABBV,112.450,N/A,269779890488,19418945818,30762984954,None,N/A,15.12,N/A,4.19,N/A,13.892613,N/A,8.769627,N/A,N/A
27,ALLE,133.660,N/A,13130209071,628098036,1228770971,None,N/A,14.6,N/A,4.43,N/A,20.904713,N/A,10.685644,N/A,N/A
30,AMCR,12.211,N/A,24843907573,1898915022,2779317050,None,N/A,3.99,N/A,1.51,N/A,13.083212,N/A,8.938853,N/A,N/A
37,ANET,329.580,N/A,21744882090,760222719,1494620539,None,N/A,7.38,N/A,10.75,N/A,28.603305,N/A,14.548764,N/A,N/A
39,ANTM,371.980,N/A,108838805917,7761650984,117194177829,None,N/A,2.75,N/A,0.7552,N/A,14.022636,N/A,0.928705,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,ULTA,336.920,N/A,18024206433,660710152,2018112768,None,N/A,9.48,N/A,3.1,N/A,27.280051,N/A,8.931219,N/A,N/A
463,USB,57.920,N/A,87898819757,6532436911,22399099532,None,N/A,1.88,N/A,3.5,N/A,13.455747,N/A,3.924212,N/A,N/A
481,WELL,75.970,N/A,33043912905,1181839907,4562473304,None,N/A,2.04,N/A,7.13,N/A,27.959720,N/A,7.242544,N/A,N/A
498,XRX,24.690,N/A,None,None,None,12.526781,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [136]:
for column in ['Price-to-earnings ratio','Enterprise value','EBITDA','Gross profit','Price-to-book ratio', 'Price-to-sales ratio','EV/EBITDA',  'EV/GP']:
    hqv_dataframe[column].fillna(hqv_dataframe[column].mean(),inplace=True)


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [137]:
hqv_dataframe[hqv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score


In [138]:
hqv_dataframe

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,A,137.53,N/A,4.148077e+10,1.292975e+09,3.081228e+09,34.651666,N/A,8.63,N/A,7.5700,N/A,32.081653,N/A,13.462415,N/A,N/A
1,AAL,23.09,N/A,4.130438e+10,-9.134634e+09,1.804147e+10,-2.975949,N/A,-2.24,N/A,0.8617,N/A,-4.521734,N/A,2.289413,N/A,N/A
2,AAP,187.02,N/A,1.310442e+10,1.048680e+09,4.537567e+09,19.230496,N/A,3.55,N/A,1.2300,N/A,12.496115,N/A,2.887984,N/A,N/A
3,AAPL,139.23,N/A,2.251326e+12,8.551336e+10,1.175935e+11,30.515620,N/A,33.83,N/A,7.7500,N/A,26.327186,N/A,19.144994,N/A,N/A
4,ABBV,112.45,N/A,2.697799e+11,1.941895e+10,3.076298e+10,23.484854,N/A,15.12,N/A,4.1900,N/A,13.892613,N/A,8.769627,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.78,N/A,4.686268e+10,1.785040e+09,4.257681e+09,30.308862,N/A,-4.60,N/A,6.4400,N/A,26.253013,N/A,11.006621,N/A,N/A
501,ZBH,175.50,N/A,4.330495e+10,1.398920e+09,5.026019e+09,22.093283,N/A,2.99,N/A,5.2100,N/A,30.955981,N/A,8.616155,N/A,N/A
502,ZBRA,508.99,N/A,2.929904e+10,7.819629e+08,2.102235e+09,32.805023,N/A,12.69,N/A,6.2300,N/A,37.468579,N/A,13.937092,N/A,N/A
503,ZION,56.59,N/A,9.493556e+09,1.313880e+09,3.148618e+09,12.860578,N/A,1.31,N/A,2.7800,N/A,7.225586,N/A,3.015150,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [139]:
for row in hqv_dataframe.index:
    hqv_dataframe.loc[row,'PE percentile'] = stats.percentileofscore(hqv_dataframe['Price-to-earnings ratio'],hqv_dataframe.loc[row,'Price-to-earnings ratio'])/100
    hqv_dataframe.loc[row,'PB percentile'] = stats.percentileofscore(hqv_dataframe['Price-to-book ratio'],hqv_dataframe.loc[row,'Price-to-book ratio'])/100
    hqv_dataframe.loc[row,'PS percentile'] = stats.percentileofscore(hqv_dataframe['Price-to-sales ratio'],hqv_dataframe.loc[row,'Price-to-sales ratio'])/100
    hqv_dataframe.loc[row,'EV/EBITDA percentile'] = stats.percentileofscore(hqv_dataframe['EV/EBITDA'],hqv_dataframe.loc[row,'EV/EBITDA'])/100
    hqv_dataframe.loc[row,'EV/GP percentile'] = stats.percentileofscore(hqv_dataframe['EV/GP'],hqv_dataframe.loc[row,'EV/GP'])/100

hqv_dataframe

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,A,137.53,N/A,4.148077e+10,1.292975e+09,3.081228e+09,34.651666,0.811881,8.63,0.742574,7.5700,0.79604,32.081653,0.867327,13.462415,0.752475,N/A
1,AAL,23.09,N/A,4.130438e+10,-9.134634e+09,1.804147e+10,-2.975949,0.035644,-2.24,0.045545,0.8617,0.069307,-4.521734,0.039604,2.289413,0.081188,N/A
2,AAP,187.02,N/A,1.310442e+10,1.048680e+09,4.537567e+09,19.230496,0.366337,3.55,0.441584,1.2300,0.134653,12.496115,0.271287,2.887984,0.114851,N/A
3,AAPL,139.23,N/A,2.251326e+12,8.551336e+10,1.175935e+11,30.515620,0.760396,33.83,0.952475,7.7500,0.80495,26.327186,0.780198,19.144994,0.906931,N/A
4,ABBV,112.45,N/A,2.697799e+11,1.941895e+10,3.076298e+10,23.484854,0.550495,15.12,0.875248,4.1900,0.564356,13.892613,0.348515,8.769627,0.508911,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.78,N/A,4.686268e+10,1.785040e+09,4.257681e+09,30.308862,0.754455,-4.60,0.043564,6.4400,0.746535,26.253013,0.776238,11.006621,0.667327,N/A
501,ZBH,175.50,N/A,4.330495e+10,1.398920e+09,5.026019e+09,22.093283,0.437624,2.99,0.374257,5.2100,0.677228,30.955981,0.857426,8.616155,0.49703,N/A
502,ZBRA,508.99,N/A,2.929904e+10,7.819629e+08,2.102235e+09,32.805023,0.8,12.69,0.851485,6.2300,0.738614,37.468579,0.885149,13.937092,0.774257,N/A
503,ZION,56.59,N/A,9.493556e+09,1.313880e+09,3.148618e+09,12.860578,0.160396,1.31,0.110891,2.7800,0.390099,7.225586,0.063366,3.015150,0.122772,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [140]:
from statistics import mean

for row in hqv_dataframe.index:
    momentum_percentiles = []
    momentum_percentiles.append(hqv_dataframe.loc[row,'PE percentile'])
    momentum_percentiles.append(hqv_dataframe.loc[row,'PB percentile'])
    momentum_percentiles.append(hqv_dataframe.loc[row,'PS percentile'])
    momentum_percentiles.append(hqv_dataframe.loc[row,'EV/EBITDA percentile'])
    momentum_percentiles.append(hqv_dataframe.loc[row,'EV/GP percentile'])
    
    hqv_dataframe.loc[row,'Robust Value score'] = mean(momentum_percentiles)
hqv_dataframe  

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,A,137.53,N/A,4.148077e+10,1.292975e+09,3.081228e+09,34.651666,0.811881,8.63,0.742574,7.5700,0.79604,32.081653,0.867327,13.462415,0.752475,0.794059
1,AAL,23.09,N/A,4.130438e+10,-9.134634e+09,1.804147e+10,-2.975949,0.035644,-2.24,0.045545,0.8617,0.069307,-4.521734,0.039604,2.289413,0.081188,0.054257
2,AAP,187.02,N/A,1.310442e+10,1.048680e+09,4.537567e+09,19.230496,0.366337,3.55,0.441584,1.2300,0.134653,12.496115,0.271287,2.887984,0.114851,0.265743
3,AAPL,139.23,N/A,2.251326e+12,8.551336e+10,1.175935e+11,30.515620,0.760396,33.83,0.952475,7.7500,0.80495,26.327186,0.780198,19.144994,0.906931,0.84099
4,ABBV,112.45,N/A,2.697799e+11,1.941895e+10,3.076298e+10,23.484854,0.550495,15.12,0.875248,4.1900,0.564356,13.892613,0.348515,8.769627,0.508911,0.569505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.78,N/A,4.686268e+10,1.785040e+09,4.257681e+09,30.308862,0.754455,-4.60,0.043564,6.4400,0.746535,26.253013,0.776238,11.006621,0.667327,0.597624
501,ZBH,175.50,N/A,4.330495e+10,1.398920e+09,5.026019e+09,22.093283,0.437624,2.99,0.374257,5.2100,0.677228,30.955981,0.857426,8.616155,0.49703,0.568713
502,ZBRA,508.99,N/A,2.929904e+10,7.819629e+08,2.102235e+09,32.805023,0.8,12.69,0.851485,6.2300,0.738614,37.468579,0.885149,13.937092,0.774257,0.809901
503,ZION,56.59,N/A,9.493556e+09,1.313880e+09,3.148618e+09,12.860578,0.160396,1.31,0.110891,2.7800,0.390099,7.225586,0.063366,3.015150,0.122772,0.169505


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [142]:
hqv_dataframe.sort_values('Robust Value score',ascending=False,inplace=True)
hqv_dataframe = hqv_dataframe[:50]
hqv_dataframe.reset_index(inplace=True, drop=True)
hqv_dataframe

<ipython-input-142-e91a68af479f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqv_dataframe.sort_values('Robust Value score',ascending=False,inplace=True)


,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,NOW,559.920,N/A,1.039186e+11,5.383409e+08,3.576268e+09,102.528469,0.980198,38.52,0.960396,23.50,0.990099,193.034905,0.99802,29.057834,0.980198,0.981782
1,DXCM,395.700,N/A,3.720241e+10,3.706527e+08,1.335707e+09,165.557903,0.992079,21.50,0.930693,20.27,0.982178,100.370005,0.992079,27.852219,0.976238,0.974653
2,IDXX,519.730,N/A,4.402644e+10,8.261888e+08,1.618776e+09,68.778531,0.952475,70.33,0.974257,16.33,0.962376,53.288593,0.938614,27.197369,0.974257,0.960396
3,MKTX,552.910,N/A,2.100553e+10,4.255758e+08,7.124504e+08,64.138954,0.930693,22.30,0.932673,30.84,0.99802,49.357902,0.928713,29.483493,0.984158,0.954851
4,NVDA,639.600,N/A,3.804031e+11,5.831945e+09,1.041591e+10,45.713297,0.887129,23.32,0.940594,23.33,0.988119,65.227479,0.958416,36.521348,0.994059,0.953663
5,ALGN,604.430,N/A,4.696463e+10,4.825350e+08,1.806724e+09,65.874960,0.940594,15.24,0.879208,19.40,0.978218,97.328952,0.988119,25.994357,0.966337,0.950495
6,ADSK,298.100,N/A,6.575288e+10,7.099381e+08,3.642392e+09,61.601319,0.922772,69.58,0.972277,17.12,0.966337,92.617773,0.982178,18.052117,0.893069,0.947327
7,ILMN,421.000,N/A,5.826832e+10,6.368043e+08,2.256420e+09,74.380887,0.964356,12.40,0.847525,18.39,0.970297,91.501131,0.980198,25.823348,0.964356,0.945347
8,MA,384.000,N/A,3.809423e+11,8.809120e+09,1.606041e+10,48.611581,0.893069,59.75,0.970297,25.53,0.994059,43.244078,0.908911,23.719333,0.958416,0.94495
9,TYL,452.980,N/A,1.740935e+10,2.016636e+08,5.455791e+08,79.293326,0.970297,9.39,0.79604,16.23,0.960396,86.328705,0.976238,31.909866,0.992079,0.93901


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [143]:
portfolio_input()

Enter the value of your portfolio: 10000000
10000000.0


In [144]:
position_size = float(portfolio_size)/len(hqv_dataframe.index)
for i in hqv_dataframe.index:
    hqv_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/hqv_dataframe.loc[i,'Price']) 
hqv_dataframe

,Ticker,Price,Number of shares to buy,Enterprise value,EBITDA,Gross profit,Price-to-earnings ratio,PE percentile,Price-to-book ratio,PB percentile,Price-to-sales ratio,PS percentile,EV/EBITDA,EV/EBITDA percentile,EV/GP,EV/GP percentile,Robust Value score
0,NOW,559.920,357,1.039186e+11,5.383409e+08,3.576268e+09,102.528469,0.980198,38.52,0.960396,23.50,0.990099,193.034905,0.99802,29.057834,0.980198,0.981782
1,DXCM,395.700,505,3.720241e+10,3.706527e+08,1.335707e+09,165.557903,0.992079,21.50,0.930693,20.27,0.982178,100.370005,0.992079,27.852219,0.976238,0.974653
2,IDXX,519.730,384,4.402644e+10,8.261888e+08,1.618776e+09,68.778531,0.952475,70.33,0.974257,16.33,0.962376,53.288593,0.938614,27.197369,0.974257,0.960396
3,MKTX,552.910,361,2.100553e+10,4.255758e+08,7.124504e+08,64.138954,0.930693,22.30,0.932673,30.84,0.99802,49.357902,0.928713,29.483493,0.984158,0.954851
4,NVDA,639.600,312,3.804031e+11,5.831945e+09,1.041591e+10,45.713297,0.887129,23.32,0.940594,23.33,0.988119,65.227479,0.958416,36.521348,0.994059,0.953663
5,ALGN,604.430,330,4.696463e+10,4.825350e+08,1.806724e+09,65.874960,0.940594,15.24,0.879208,19.40,0.978218,97.328952,0.988119,25.994357,0.966337,0.950495
6,ADSK,298.100,670,6.575288e+10,7.099381e+08,3.642392e+09,61.601319,0.922772,69.58,0.972277,17.12,0.966337,92.617773,0.982178,18.052117,0.893069,0.947327
7,ILMN,421.000,475,5.826832e+10,6.368043e+08,2.256420e+09,74.380887,0.964356,12.40,0.847525,18.39,0.970297,91.501131,0.980198,25.823348,0.964356,0.945347
8,MA,384.000,520,3.809423e+11,8.809120e+09,1.606041e+10,48.611581,0.893069,59.75,0.970297,25.53,0.994059,43.244078,0.908911,23.719333,0.958416,0.94495
9,TYL,452.980,441,1.740935e+10,2.016636e+08,5.455791e+08,79.293326,0.970297,9.39,0.79604,16.23,0.960396,86.328705,0.976238,31.909866,0.992079,0.93901


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [145]:
writer = pd.ExcelWriter('value_strategy.xlsx',engine='xlsxwriter')
hqv_dataframe.to_excel(writer,sheet_name='Value Strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [146]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [153]:
column_format = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of shares to buy',integer_template],
    'D':['Enterprise value',integer_template],
    'E':['EBITDA',integer_template],
    'F':['Gross profit',integer_template],
    'G':['Price-to-earnings ratio',string_template],
    'H':['PE percentile',string_template],
    'I':['Price-to-book ratio',string_template],
    'J':['PE percentile',string_template],
    'K':['Price-to-book ratio',string_template],
    'L':['PB percentile',string_template],
    'M':['EV/EBITDA',string_template],
    'N':['EV/EBITDA percentile',string_template],
    'O':['EV/GP',string_template],
    'P':['EV/GP percentile',string_template],
    'Q':['Robust Value score',string_template],
}

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [154]:
for column in column_format.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}',25,column_format[column][1])    
    writer.sheets['Value Strategy'].write(f'{column}1',column_format[column][0],string_template)
writer.save()